In [8]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F
import torch
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import mat4py
import pandas as pd
import data_tracks
import models
from matplotlib_venn import venn3, venn2
from sklearn.model_selection import KFold
from colorama import Fore
import pickle as pkl
import viz
from style import *
import analyze_helper
import train
from sklearn import decomposition
from sklearn.calibration import calibration_curve

outcome_def = 'y_consec_thresh'
out_dir = 'results/feb23_2'
results = analyze_helper.load_results(out_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# compare all models
**baseline stats**

In [9]:
viz.print_metadata(acc=results.accuracy.max())

valid:		2066.0 aux+ / 7594 (0.272)
----------------------------------------
no_hotspots:	1735.0 aux+ / 7245 (0.239)
----------------------------------------
lifetime<=26:	4720 aux+ / 5127 (0.921)
lifetime>=145:	  62 aux- /   67 (0.925)
remaining:	1266 aux+ / 2051 (0.617)
----------------------------------------
predicted acc:			  0.674
total acc:			  0.851


**look at prediction metrics**

In [10]:
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r = r.sort_values(by=['accuracy', 'balanced_accuracy'], ascending=False).head(10)
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])

,accuracy,balanced_accuracy,f1,precision,recall,roc_auc
model_type,,,,,,
rf_16_select_rf=3_ros=1.2_h=1,0.674,0.651,0.741,0.723,0.771,0.693
rf_16_select_rf=3_ros=1.2_h=0,0.673,0.65,0.741,0.722,0.773,0.696
rf_16_select_rf=3_ros=1_h=1,0.672,0.652,0.738,0.726,0.76,0.695
logistic_35_select_rf=3_ros=1.2_h=0,0.668,0.665,0.723,0.745,0.718,0.709
rf_16_select_rf=3_ros=1_h=0,0.668,0.65,0.734,0.724,0.755,0.695
rf_16_select_rf=3_ros=1.2_h=-1,0.668,0.646,0.735,0.721,0.762,0.69
logistic_25_select_rf=3_ros=1.2_h=0,0.665,0.663,0.721,0.744,0.716,0.698
rf_16_select_rf=3_ros=0.8_h=0,0.664,0.648,0.726,0.726,0.736,0.693
rf_16_select_rf=3_ros=0.8_h=1,0.661,0.647,0.724,0.725,0.732,0.696


**look at feat importances**

In [7]:
r = results
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
# r = r[r.accuracy > np.percentile(r.accuracy, 80)]


keys = [k for k in r if '_f' in k]
keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})
# r = r[r2.index.str.contains('ros')]
# r = r[r.index.str.contains('none')]
r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r
# 
r = rank(r)
# r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank

idxs = r.index
r.insert(0, 'acc', results.loc[idxs]['accuracy'])
r = r.sort_values('acc', ascending=False)
r = r.reindex(r.iloc[0].sort_values(ascending=True).index, axis=1) # sort cols by mean rank
subset = list(r.keys())
subset.remove('acc')
r.insert(1, 'num_feats', r.apply(lambda row: np.sum(~np.isnan(row.values)) - 1, axis=1))


def color_vals(val):
    colors = {
        1: cb, 2: cp, 3: cr,
    }
    if not val in colors:
        color = 'maroon'
    else:
        color = colors[val]
    return 'color: %s' % color

r = r[r.num_feats <= 5].head(10)

r.fillna("").style.applymap(color_vals, subset=subset)
# r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1, subset=subset) # rows differently colored

,acc,num_feats,fall,slope_end,fall_extended,fall_late_extended,fall_local_11,lifetime,msd,td
model_type,,,,,,,,,,
svm_16_ros=1.2_select_rf=3,0.705,2,1,2,,,,,,
mlp2_16_ros=1.2_select_rf=3,0.697,2,1,2,,,,,,
mlp2_35_ros=1.2_select_rf=3,0.694,2,,,2,1,,,,
svm_35_ros=1.2_select_rf=3,0.692,2,,,1,2,,,,
svm_16_ros=1_select_rf=3,0.689,2,1,2,,,,,,
mlp2_35_ros=1_select_rf=3,0.688,2,,,2,1,,,,
mlp2_16_ros=1_select_rf=3,0.688,2,1,2,,,,,,
svm_35_ros=1_select_rf=3,0.688,2,,,2,1,,,,
svm_25_ros=1.2_select_rf=3,0.686,3,,,1,2,3,,,


# analyze model predictions

In [4]:
df = data_tracks.get_data()
n = df.shape[0]

# normalize and store
def normalize(df):
    X = df[data_tracks.get_feature_names(df)]
    X_mean = X.mean()
    X_std = X.std()
    ks = list(X.keys())
    norms = {ks[i]: {'mu': X_mean[i], 'std': X_std[i]} for i in range(len(ks))}
    X = (X - X_mean) / X_std
    y = df[outcome_def].values
    return X, y, norms
X, y, norms = normalize(df)



# split testing data based on cell num
# cv_idx = data_tracks.cell_nums_train[0]
# idxs_test = df.cell_num.isin([cv_idx]) # this is the cv set for model's trained in the first cv fold
# X_test, Y_test = X[idxs_test], y[idxs_test]

# look at best models
r = results
r = r.sort_values('accuracy', ascending=False)

idx = np.array(r.index)
accs = np.array(r.accuracy)
for i in range(5):
    print(f'{accs[i]:.3f}', idx[i])
# model_name = 'svm_15_ros=1.2_select_rf=3' # idx[0]
# model_name = idx[0]
model_name = 'svm_16_ros=1.2_select_rf=3'

0.722 svm_25_ros=1.2_None=3
0.719 mlp2_16_ros=1.2_select_rf=15
0.719 rf_35_ros=1_None=3
0.718 svm_35_ros=1.2_None=3
0.718 rf_35_ros=1.2_None=3


### look at single model

In [5]:
d_full_cv, idxs_cv = analyze_helper.get_data_over_folds(model_name, out_dir, df.cell_num, X, y)
y_full_cv = df[outcome_def].iloc[idxs_cv].values.astype(np.int)
preds = d_full_cv[model_name].values
preds_proba = d_full_cv[model_name + '_proba'].values

results_individual = pkl.load(open(oj(out_dir, f'{model_name}.pkl'), 'rb'))
assert np.sum(idxs_cv == np.arange(idxs_cv.size)) == idxs_cv.size, \
       'points not in same order'
assert np.mean(preds==y_full_cv) == np.average(results_individual['cv']['accuracy'], 
                                               weights=results_individual['num_pts_by_fold_cv']), \
        'did not properly load model/data'

In [ ]:
num_to_plot = 16
tp = np.logical_and(preds==1, y_full_cv==1)
tn = np.logical_and(preds==0, y_full_cv==0)
fp = preds > y_full_cv
fn = preds < y_full_cv

print('total pts', preds.shape[0])
# for idxs, name in zip([fp, fn, tp, tn], ['fp', 'fn', 'tp', 'tn']):
# for idxs, name in zip([fp, fn], ['fp', 'fn']):
for idxs, name in zip([tp, tn], ['tp', 'tn']):
    print(name, idxs.sum(), idxs.sum() / preds.shape[0])
    inds = viz.viz_biggest_errs(df, idxs_cv, idxs, y_full_cv, preds, preds_proba, num_to_plot)
#     plt.savefig(f'{name}.pdf')
    plt.show()

In [196]:
# calculate hotspots acc
results_individual = pkl.load(open(oj(out_dir, f'{model_name}.pkl'), 'rb'))
df = data_tracks.get_data(use_processed=False, remove_hotspots=False, save_processed=False)
df = df[df['hotspots']==1]
X, y, norms = normalize(df)

preds, pred_proba = analyze_helper.analyze_individual_results(results_individual, 
                                               X, y, print_results=False, 
                                               plot_results=False, model_cv_fold=1)
print(f'hotspots acc:\t\t\t{np.mean(preds==df[outcome_def])}')

loading + preprocessing data...
	loading tracks...
	preprocessing data...
	adding features...
hotspots acc:			0.6995884773662552


In [ ]:
dist_to_end = df.iloc[idxs_fp]['lifetime'] - df.iloc[idxs_fp]['Y_peak_idx'] - 1
frac_to_end = 1 - df.iloc[idxs_fp]['Y_peak_idx'] / df.iloc[idxs_fp]['lifetime']
print(np.unique(dist_to_end, return_counts=True))
plt.hist(frac_to_end, bins=100)
plt.show()

In [33]:
print('inter-cell accs', np.array(results_individual['cv']['accuracy']).round(2))

inter-cell accs [0.71 0.73 0.69 0.69 0.69]


In [ ]:
viz.jointplot_grouped('fall', 'slope_end', 'y_consec_sig', df, scatter_alpha=0.1)

In [ ]:
# m = results_individual['imps']['model'][0]    
# viz.plot_decision_boundary('X_std', 'fall', m, df, norms, num_pts=df.shape[0])

In [ ]:
key = 'X_max' # lifetime, X_min, fall, X_max
viz.viz_errs_1d(X.iloc[idxs_cv], preds, preds_proba, y_full_cv, norms, key=key)

In [ ]:
key1 = 'fall' # fall, x_pos, pc1
key2 = 'X_mean' # X_min, y_pos, pc2
viz.viz_errs_2d(df, idxs_cv, preds, y_full_cv, key1=key1, key2=key2)
plt.xscale('log')
plt.yscale('log')

In [ ]:
print(results_individual['feat_names_selected'])
d = df.iloc[idxs_cv]
d['preds'] = preds
d['preds_proba'] = preds_proba
key1 = 'fall'
key2 = 'slope_end'
plt.figure(dpi=300)
plt.scatter(d[key1], d[key2], c=d['preds_proba'], alpha=0.5)
plt.xlabel(key1)
plt.ylabel(key2)
cb = plt.colorbar()
cb.set_label('predicted prob.')

In [ ]:
viz.viz_errs_outliers_venn(X.iloc[idxs_cv], preds, y_full_cv, num_feats_reduced=5)

In [ ]:
plt.figure(dpi=200)
prob_true, prob_pred = calibration_curve(y_full_cv, preds_proba, 
                                         normalize=False, n_bins=10, strategy='quantile')
plt.plot(prob_true, prob_pred, '.-')
plt.plot([0, 1], [0, 1], alpha=0.3, color='gray')
plt.xlabel('true prob')
plt.ylabel('predicted prob')
plt.show()

**look at pcs**

In [ ]:
# perform pca
pca = decomposition.PCA(whiten=True)
pca.fit(X.iloc[idxs_cv])
viz.plot_pcs(pca, X.iloc[idxs_cv])

In [ ]:
pca = decomposition.PCA(n_components=2, whiten=True)
X_reduced = pca.fit_transform(X.iloc[idxs_cv])
plt.figure(dpi=200)
ys_neg = y[idxs_cv] == 0
plt.plot(X_reduced[:, 0][ys_neg], X_reduced[:, 1][ys_neg], 'o', 
         color=cr, alpha=0.3, markeredgewidth=0, ms=2)
plt.plot(X_reduced[:, 0][~ys_neg], X_reduced[:, 1][~ys_neg], 'o', 
         color=cb, alpha=0.3, markeredgewidth=0, ms=2)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

### look at many models

In [201]:
# model_names = ['mlp2_11_none', 'svm_35_none', 'logistic_4_none', 'rf_9_none']
# model_names = ['mlp2_11_none', 'mlp2_9_none', 'mlp2_4_none']
model_names = idx[:3]
d_full_cv, idxs_cv = analyze_helper.get_data_over_folds(model_names, out_dir, df.cell_num, X, y)
y_full_cv = df[outcome_def].iloc[idxs_cv].values

**ensemble err**

In [202]:
# ensemble
d_full_cv_probs = d_full_cv[[k for k in d_full_cv.keys() if 'proba' in k]]
preds_soft = d_full_cv_probs.sum(axis=1) / d_full_cv_probs.shape[1]

for score_name in train.scorers.keys():
    print(score_name, train.scorers[score_name](y_full_cv, preds_soft > 0.5))
# metrics.accuracy_score(y_ensemble, preds_soft>0.5) 

balanced_accuracy 0.7042086967746695
accuracy 0.7206502107164359
precision 0.7748538011695907
recall 0.7733463035019456
f1 0.7740993184031157
roc_auc 0.7042086967746695
precision_recall_curve (array([0.61890427, 0.7748538 , 1.        ]), array([1.       , 0.7733463, 0.       ]), array([False,  True]))
roc_curve (array([0.        , 0.36492891, 1.        ]), array([0.       , 0.7733463, 1.       ]), array([2, 1, 0]))


**venn-diagram**

In [ ]:
sets = []
for model_name in model_names:
    args = np.argwhere(d_full_cv[model_name] != y_full_cv)
    sets.append(set(args.flatten().tolist()))
    
plt.figure(dpi=300)
plt.title('venn diagram of shared errors')
venn3(sets, model_names[:3])
plt.show()